## Exercise I

The input is a textual csv file containing the daily value of PM10 for a set of sensors, and in each line of the files has the following format:
```sensorId,date,PM10 value (μg/m3)\n```

Here is the example of data:
```
s1,2016-01-01,20.5
s2,2016-01-01,30.1
s1,2016-01-02,60.2
s2,2016-01-02,20.4
s1,2016-01-03,55.5
s2,2016-01-03,52.5
```

You're required to use pyspark to load the file, filter the values and use map/reduce code idea to give the output. The output is a line for each sensor on the standard output.
Each line contains a `sensorId` and the list of `dates` with a PM10 values greater than 50 for that sensor. The example output:
```
(s1, [2016-01-02, 2016-01-03])
(s2, [2016-01-03])
```



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_list

# Create a SparkSession
spark = SparkSession.builder.appName("PM10Filter").getOrCreate()

# Load the CSV file
df = spark.read.csv("sensors.csv", header=False)

# Rename columns
df = df.withColumnRenamed("_c0", "sensorId") \
       .withColumnRenamed("_c1", "date") \
       .withColumnRenamed("_c2", "PM10")

# Filter values greater than 50
filtered_df = df.filter(col("PM10") > 50)

# Group by sensorId and collect dates
result_df = filtered_df.groupBy("sensorId").agg(collect_list("date").alias("dates"))

# Show the result
result_df.show(truncate=False)

# Stop the SparkSession
spark.stop()


## Exercise II

Using the same data of the Exercise I, you're required to get the output: sensors ordered by the number of critical days. Each line of the output contains the number of days with a PM10 values greater than 50 for a sensor `s` and the `sensorId` of sensor `s`.

The example of the output:
```
2, s1
1, s2
```



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, size

# Create a SparkSession
spark = SparkSession.builder.appName("CriticalDays").getOrCreate()

# Load the CSV file
df = spark.read.csv("sensors.csv", header=False)

# Rename columns
df = df.withColumnRenamed("_c0", "sensorId") \
       .withColumnRenamed("_c1", "date") \
       .withColumnRenamed("_c2", "PM10")

# Filter values greater than 50
filtered_df = df.filter(col("PM10") > 50)

# Group by sensorId and count number of critical days
result_df = filtered_df.groupBy("sensorId").agg(size(collect_list("date")).alias("num_critical_days"))

# Order by number of critical days
result_df = result_df.orderBy("num_critical_days", ascending=False)

# Show the result
result_df.show(truncate=False)

# Stop the SparkSession
spark.stop()


## Exercise III

In this exercise, you're given an input: A CSV file containing a list of profiles

- Header: `name,age,gender`
- Each line of the file contains the information about one user

The example of input data
```
name,surname,age
Paolo,Garza,42
Luca,Boccia,41
Maura,Bianchi,16
```

You're required to use pyspark to load and analyze the data to achieve the output: A CSV file containing one line for each profile. The original age attribute is substituted with a new attributed called rangeage of type String.
```
rangeage = "[" + (age/10)*10 + "-" + (age/10)*10+9 + "]"
```





In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, lit, floor

# Create a SparkSession
spark = SparkSession.builder.appName("ageRange").getOrCreate()

# Load the CSV file
df = spark.read.csv("profiles.csv", header=True)

# Define the transformation
df = df.withColumn("rangeage",
                   concat(
                       lit('['),
                       (floor(df["age"]/10)*10).cast("string"),
                       lit('-'),
                       ((floor(df["age"]/10)*10)+9).cast("string"),
                       lit(']')
                   )
                  )

# Drop the original 'age' column
df = df.drop('age')

# Write the result back to a new CSV file
df.write.csv("profiles_with_rangeage.csv", header=True)

# Stop the SparkSession
spark.stop()
